In [2]:
import numpy as np
import pandas as pd
import glob
import ipyparallel as p
import os
import ntpath

In [3]:
exname = 'smex3'

In [4]:
files = glob.glob('/Volumes/data/RHESSys_out/Vcm/'+exname+'/*_basin.daily')

In [8]:
files = pd.DataFrame({'basin_daily':files})

In [9]:
def getIDX(fl):
    return int(fl.split('_')[-2])

In [10]:
files['idx'] = files.basin_daily.map(getIDX)

In [11]:
c = p.Client()
view = c.load_balanced_view()

In [12]:
c.ids

[0, 1, 2, 3, 4, 5, 6, 7]

In [13]:
%%px
import os

def wateryear(index):
    year = index.year
    strt = pd.datetime(int(year),10,1) # start of the next water year+1
    
    if index<strt:
        wyear = year
    elif index>=strt: 
        wyear = year+1

    return wyear

def readRHESSysBasin(filename):
        
    if os.path.isfile(filename) == False:
        print 'Input file does not exist'
    else:
        bd = filename # basin output path

        bddat = pd.read_table(bd,delim_whitespace=True,
                               header='infer', parse_dates={'foo':[2,1,0]}, index_col='foo')
        bddat.reset_index(inplace=True)
        #bddat.index = pd.DatetimeIndex(bddat['foo'])

        data = bddat
        del data['foo']
        
        #data['wateryear'] = data.index.map(wateryear) # calculate the water year
        
        # calculate accumulated precip and snowfall by water year
        
        data['ET'] = data.evap+data.trans
        
        return data

In [14]:
def readRHESSysBasin(filename):
        
    if os.path.isfile(filename) == False:
        print 'Input file does not exist'
    else:
        bd = filename # basin output path

        bddat = pd.read_table(bd,delim_whitespace=True,
                               header='infer', parse_dates={'foo':[2,1,0]}, index_col='foo')
        bddat.reset_index(inplace=True)
        #bddat.index = pd.DatetimeIndex(bddat['foo'])

        data = bddat
        del data['foo']
        
        #data['wateryear'] = data.index.map(wateryear) # calculate the water year
        
        # calculate accumulated precip and snowfall by water year
        
        data['ET'] = data.evap+data.trans
        
        return data

In [15]:
#x = params.iloc[60000]

#bd = x.basin_daily
#patchd = x.patch_daily
#idx = x.idx
#strt = x.date_peakSWE
#nd = x.date_ONS

def checkSF(idx,bd):
    dr = pd.date_range('2011-10-1','2012-10-2',freq='D')
    data = pd.read_table(bd, delim_whitespace=True)
    data.index = pd.DatetimeIndex(dr)
    
    return idx,data.streamflow.sum()
    
def processSM(bd,patchd,idx,strt,nd):
    tmp = np.NaN
    dr = pd.date_range('2011-10-1','2012-10-2',freq='D')
    
    lt = len(dr)
    
    # read in the basin daily data
    data = pd.read_table(bd, delim_whitespace=True)
    
    if len(data) < lt:
        return idx,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp
    else:
        data.index = pd.DatetimeIndex(dr)

        data = data.loc[strt:nd] # crop the data to the melt season

        data['ET'] = data.evap + data.trans

        ET = data.ET.sum() # compute total ET
        Q = data.streamflow.sum() # compute total streamflow 
        Qbf = data.baseflow.sum() # compute total baseflow
        Qrf = data['return'].sum() # compute total returnflow,p
        rech = data.recharge.sum() # compute total recharge
        sat = data['%sat_area'].mean() # compute the mean saturated area
        p = data.precip.sum() # compute water year precip
        rz_storage = data.rz_storage.mean() # root zone storage
        rz_drainage = data.rz_drainage.mean() # root zone drainage 
        unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
        unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
        sat_def = data.sat_def.mean() # saturation deficite (mm)
        sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?

    # read in the patch data

    data = pd.read_table(patchd, delim_whitespace=True) # load the patch data
    
    if len(data) < lt:
        return idx,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp
    else:
        data.index = pd.DatetimeIndex(dr)
        data = data.loc[strt:nd] # crop the data to the melt season

        Qin = data.Qin.mean() # saturated flow in
        Qout = data.Qout.mean() # saturated flow out
        streamflow = data.streamflow.mean() # streamflow
        rz_field_capacity = data.rz_field_capacity.mean() # mean field capacity
        snow_melt = data.snow_melt.sum()
    
        return idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,Qin,Qout,streamflow,rz_field_capacity,snow_melt


#print idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,idx,Qin,Qout,streamflow,rz_field_capacity,snow_melt

In [16]:
#res = view.map(processSM,params.basin_daily[0:1000],params.patch_daily[0:1000],params.idx[0:1000],params.date_peakSWE[0:1000],params.date_ONS[0:1000])
res = view.map(checkSF,files.idx,files.idx)

In [44]:
print checkSF(files.idx[ct],files.basin_daily[ct])
ct += 1

(100555, 0.0)


In [73]:
data = pd.read_table(files.basin_daily[ct], delim_whitespace=True)
print data.snowmelt.sum()
ct += 1

0.0


In [51]:
data.snowmelt

0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25     0
26     0
27     0
28     0
29     0
      ..
338    0
339    0
340    0
341    0
342    0
343    0
344    0
345    0
346    0
347    0
348    0
349    0
350    0
351    0
352    0
353    0
354    0
355    0
356    0
357    0
358    0
359    0
360    0
361    0
362    0
363    0
364    0
365    0
366    0
367    0
Name: snowmelt, dtype: float64

In [36]:
res.ready()

True

In [20]:
res.progress

101250

In [22]:
idx,SF = zip*(res.result)

CompositeError: one or more exceptions from call to method: checkSF
[1:apply]: IOError: Expected file path name or file-like object, got <type 'numpy.int64'> type
[2:apply]: IOError: Expected file path name or file-like object, got <type 'numpy.int64'> type
[5:apply]: IOError: Expected file path name or file-like object, got <type 'numpy.int64'> type
[0:apply]: IOError: Expected file path name or file-like object, got <type 'numpy.int64'> type
.... 101246 more exceptions ...

In [ ]:
data = pd.DataFrame

In [91]:
idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,Qin,Qout,streamflow,rz_field_capacity,snow_melt = zip(*res.result)

In [92]:
np.savez_compressed('./jemez/data/'+exname+'_sm_season.npz',idx=idx,ET=ET,Q=Q,Qbf=Qbf,Qrf=Qrf,rech=rech,sat=sat,p=p,rz_storage=rz_storage,rz_drainage=rz_drainage,unsat_storage=unsat_storage,unsat_drainage=unsat_drainage,sat_def=sat_def,sat_def_z=sat_def_z,Qin=Qin,Qout=Qout,streamflow=streamflow,rz_field_capacity=rz_field_capacity,snow_melt=snow_melt)

## Merge processing results with parameters dataset

In [93]:
params = pd.read_pickle('./jemez/data/params_Vcm_smex_method3.pcl') # load parameters dataset

In [94]:
# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,Qin,Qout,streamflow,rz_field_capacity,snow_melt]
keys = ['idx','ET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage','unsat_storage','unsat_drainage','sat_def','sat_def_z','Qin','Qout','streamflow','rz_field_capacity','snow_melt']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

In [95]:
params = pd.merge(params,proc,on='idx') # join on file index number
params.dropna(inplace=True) # drop missing values

In [96]:
params.to_hdf('./jemez/data/'+exname+'.hdf','df',format='fixed',complevel=5,complib='bzip2',fletcher32=True)

In [38]:
tmp = pd.read_hdf('./data/smex3.hdf','df')